# Real-Time Intelligent Systems Final Project Client
## March 18, 2021
#### Glory Scheel
#### Arnav Tayal

## Import packages

In [14]:
import pandas as pd 
import numpy as np 
from matplotlib import pyplot as plt
from sklearn import model_selection, preprocessing, feature_selection, ensemble, linear_model, metrics, decomposition
from sklearn.preprocessing import MinMaxScaler, normalize, StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Flatten
import socket 
import sys 
import pandas as pd 
import numpy as np 
import json

## Create function that connects to host and port and pulls training data into a list

In [41]:
def get_training_data(): 
    HOST = '127.0.0.1'  # The server's hostname or IP address
    PORT = 9998        # The port used by the server
    
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.connect((HOST, PORT))
    n = 0 
    data = []
    max_rows = 8143
    
    while True: 
        rec_data = s.recv(1024)
        row = json.loads(rec_data)
        
        print ("Number of datapoints: {}".format(n))
        
        if (rec_data == None or n == max_rows): 
            print ("Data Retreival Complete!")
            return data
        
        data.append(row)
        n += 1

## Create function that converts data types as necessary and drops unused variables

In [42]:
def process_data(data_list): 
    df = pd.DataFrame(data_list)
    cols = df.columns
    
    for col in cols: 
        if col == "date": 
            df[col] = df[col].apply(pd.to_datetime)
        elif col == 'Occupancy': 
            df[col] = df[col].apply(lambda x: str(x))
        else:
            df[col] = df[col].apply(pd.to_numeric)
    
    df.drop(columns = ['Humidity', 'number', 'date'], inplace = True)
    return df

## Create function that first collects training data then processes it using previous functions created

In [43]:
def train_and_collect(): 
    data = get_training_data()
    df = process_data(data)
    return (df)


## Run train_and_collect function to collect and preprocess dataset then train on neural network

In [77]:
df=train_and_collect()

df.drop(columns = ['Humidity', 'number', 'date'], inplace = True)
scaler=StandardScaler()
x=df.drop(columns='Occupancy')
y=df[['Occupancy']]
x_train=scaler.fit_transform(x)

X=np.array(x_train)
Y=np.array(y)

X=np.reshape(X, (X.shape[0], 1, X.shape[1]))
NNmodel=Sequential()
NNmodel.add(Dense(10, activation='relu'))
NNmodel.add(Flatten())
NNmodel.add(Dense(1,activation='sigmoid'))
NNmodel.compile(loss='binary_crossentropy', optimizer='adam',metrics=["accuracy"])

NNmodel.fit(X, Y, batch_size=64, epochs=10)

Epoch 1/10
8143/8143 [==============================] - 1s 174us/step - loss: 0.4897 - accuracy: 0.8267
Epoch 2/10
8143/8143 [==============================] - 1s 67us/step - loss: 0.2661 - accuracy: 0.9476
Epoch 3/10
8143/8143 [==============================] - 0s 56us/step - loss: 0.1581 - accuracy: 0.9718
Epoch 4/10
8143/8143 [==============================] - 0s 59us/step - loss: 0.1086 - accuracy: 0.9844
Epoch 5/10
8143/8143 [==============================] - 1s 70us/step - loss: 0.0842 - accuracy: 0.9864
Epoch 6/10
8143/8143 [==============================] - 1s 71us/step - loss: 0.0710 - accuracy: 0.9875
Epoch 7/10
8143/8143 [==============================] - 0s 58us/step - loss: 0.0634 - accuracy: 0.9877
Epoch 8/10
8143/8143 [==============================] - 0s 56us/step - loss: 0.0589 - accuracy: 0.9883
Epoch 9/10
8143/8143 [==============================] - 0s 61us/step - loss: 0.0559 - accuracy: 0.9886
Epoch 10/10
8143/8143 [==============================] - 1s 62us/step - 

## Create functions that:
   - Converts strings to json 
   - Pulls data from testing port, pre-processes data and then uses neural network to make predictions on streaming in data

In [82]:
import pandas as pd
def convertStringToJSON(st):
    return json.dumps(st)


# Function to connect to testing server, get test row, make prediction, and send result back.
def test_model(): 
    HOST = '127.0.0.1'  # The server's hostname or IP address
    PORT = 9999        # The port used by the server
    
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.connect((HOST, PORT))
    while True: 
        data = s.recv(1024)
        #print(data)
        row = json.loads(data)

        del row['Occupancy']
        del row['Humidity']
        del row['Number']
        del row['date']


        row_list = []
        
        row_list.append(row)

        df=pd.DataFrame(row_list)
        x_test=scaler.transform(df)
        Xtest=np.array(x_test)
        
        Xtest=np.reshape(Xtest, (Xtest.shape[0],1,Xtest.shape[1]))
        
        prediction = NNmodel.predict(Xtest).round()
        send_str = convertStringToJSON({"Occupancy": int(prediction)}).encode("utf-8")
        s.send(send_str)
        
        print ("Row to predict on: {}".format(row_list))
        print ("Prediction: {}".format(str(prediction)))
        



test_model()

Row to predict on: [{'Temperature': '23.7', 'Light': '585.2', 'CO2': '749.2', 'HumidityRatio': '0.00476416302416414'}]
Prediction: [[1.]]
Row to predict on: [{'Temperature': '23.718', 'Light': '578.4', 'CO2': '760.4', 'HumidityRatio': '0.00477266099212519'}]
Prediction: [[1.]]
Row to predict on: [{'Temperature': '23.73', 'Light': '572.666666666667', 'CO2': '769.666666666667', 'HumidityRatio': '0.00476515255246541'}]
Prediction: [[1.]]
Row to predict on: [{'Temperature': '23.7225', 'Light': '493.75', 'CO2': '774.75', 'HumidityRatio': '0.00474377335599685'}]
Prediction: [[1.]]
Row to predict on: [{'Temperature': '23.754', 'Light': '488.6', 'CO2': '779', 'HumidityRatio': '0.00476659399998615'}]
Prediction: [[1.]]
Row to predict on: [{'Temperature': '23.76', 'Light': '568.666666666667', 'CO2': '790', 'HumidityRatio': '0.00477933243163454'}]
Prediction: [[1.]]
Row to predict on: [{'Temperature': '23.73', 'Light': '536.333333333333', 'CO2': '798', 'HumidityRatio': '0.00477613633274892'}]
Pre

KeyboardInterrupt: 